# Naive Bayes
Naive Bayes is a classical ML algorithm used for text analytics and general classification.
The following implementation describes the Gaussian Naive Bayes.

## The Algorithm
The algorithm it is very statistical based using prior and posterior probabilities of the classes in the dataset.

Using the Bayes' Theorem below as the main idea:

$$P(A | B)P(B) = P(A \cap B) = P(B \cap A) = P(B | A)P(A)$$

$$P(A | B) = \frac{P(B | A)P(A)}{P(B)}$$


Now using the theorem we can ask what is the probability of a given class given that a specific document happened.

$$p(\text{class} \mid \mathbf {\text{data}} )={\frac {p(\mathbf {\text{data}} \mid \text{class}) * p(\text{class})}{p(\mathbf {\text{data}} )}}$$

where:


* $p(class | data)$ is called the posterior.
* $p(data | class)$ is called the likelihood.
* $p(class)$ is called the prior.
* $p(data)$ is called the marginal probability.


The equation above describes the full Bayes algorithm but some probabilites are very non pratical to calculate an example of this is the $P(data)$ because if we have a never seen document in the dataset this $P(data)$ is going to be $0$, so usually we don't calculate the marginal probabilities in real word cases.






**The term Naive comes from assuming that the variables are independent of each other when they may not be**

Then using the independance factor the Bayes' Theorem can be calculated as below:
<div>
<img src="images/BayesSimple.png" width="600"/>
</div>

## Steps
In this notebook I'm going to approach the 20 News Group dataset, the steps to solve/classify this dataset are pre-processing the text data, training the Gaussian Bayes Classifier and testing it with unseen data.

## Pre-processing
* converting all letters to lower or upper case
* converting numbers into words or removing numbers
* removing punctuations, accent marks and other diacritics
* removing white spaces
* expanding abbreviations
* removing stop words, sparse terms, and particular words
* text canonicalization

In [1]:
# Loading the Data using sklearn
from sklearn.datasets import fetch_20newsgroups
train_data = fetch_20newsgroups(subset='train', shuffle=True)
test_data = fetch_20newsgroups(subset='test', shuffle=True)

#train.data: holds the text data
#train.target: holds the id's for the classes
#train.target_names: holds the class string names
#train.filenames: holds the filenames

# Steps

* converting all letters to lower or upper case
* converting numbers into words or removing numbers
* removing emails
* removing punctuations, accent marks and other diacritics
* removing white spaces
* expanding abbreviations

* removing stop words, sparse terms, and particular words
* text canonicalization

In [2]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

punctuation = """\!\"\#\$\%\&\'\(\)\*\+\,\-\.\/\:\;\<\=\>\?[\]\^\_\`\{\|\}~"""

# Defining cleaning regexes
number_re = re.compile(r'(\d+)', re.I | re.M | re.U)
punkt_re = re.compile(r'([%s])' % punctuation, re.I | re.M | re.U)
email_re = re.compile(r'(\w+\@\w+)', re.I | re.M | re.U)
whitespaces_re = re.compile(r'(\s)', re.I | re.M | re.U)


def preprocess(doc):
    if type(doc) != str:
        raise Exception('Doc is not text data')

    # Making a copy of the original document
    _doc = doc

    # Stripping
    _doc = _doc.strip()

    # lower case
    _doc = _doc.lower()

    # removing numbers
    _doc = number_re.sub('', _doc)

    # removing punkt before email so the email regex is simpler
    _doc = punkt_re.sub('', _doc)

    # removing emails
    _doc = email_re.sub('', _doc)

    # removing long white spaces to single spacew
    _doc = whitespaces_re.sub(' ', _doc)

    # Removing stopwords and words that aren't four units long
    tokens = [
        token for token in word_tokenize(_doc)
        if (token not in stop_words and len(token) > 4 and '\\' not in token)
    ]

    return tokens

# Naive Bayes(Gaussian) Implementation

Following sections will be about the implementation of the GNB for text classification.



## Training

In the training process we are going to receive the non cleaned text data and preprocess it and build the dictionary and calculate the prior probabilities and the likelihood probabilities also all of this using the Gaussian distribution to solve the zero frequency problem assuming variable independance.




In [3]:
import numpy as np
import math

class GNB:
    _dict = set()

    def __init__(self):
        pass

    def dictionary(self, docs):
        # Building or dictionary with the entire training dataset
        [self._dict.update(preprocess(doc)) for doc in docs]

        print('Dictionary length %d\n' % (len(self._dict)))

    # Calculates p(x | y), p of x given that y happened using an gaussian distribution
    def likelihood_prob(self, x, mean_y, var_y):
        
        # Probability density function
        exponent = math.exp(-(math.pow(x-mean_y,2)/(2*math.pow(var_y,2))))
        
        return (1 / (math.sqrt(2*math.pi) * var_y)) * exponent
        #p_x_y = (1 / np.sqrt(2 * np.pi * var_y)) * np.exp(
        #    ((mean_y - x)**2) / 2 * var_y)
        
        #return p_x_y

    def prior_probs(self, X):
        counts = dict(zip(*np.unique(X, return_counts=True)))

        total = len(X)

        prior = np.array([counts[i] / total for i in range(len(counts))])

        return prior
    
    def create_table(self, X, _dict):
        # Creating the frequency table
        nrows = len(X.data)
        freq = {key: np.zeros(nrows) for key in self._dict}

        for doc, i in zip(X.data, range(nrows)):
            if(i % 1000 == 0):
                print('Doc [%d] out of [%d]' % (i, nrows))

            # Preprocessing the data
            words = preprocess(doc)

            for w in words:
                if w not in _dict:
                    continue
                # Getting the vector assigned by the word w
                vec = freq[w]

                # In the ith position (observation id) sum one of ocurrence
                vec[i] += 1

        return freq
    
    def train(self, X):
        # Creating the dictionary
        self.dictionary(X.data)

        # Calculating the class prior probabilities
        self.p_class = self.prior_probs(X.target)
        
        self.freq = self.create_table(X, self._dict)
        
        # Assigning the classes
        self.freq.update({'class': X.target})

        # class       w1                w2  w3  w4 w5 ...
        # 0     (mean_w1, var_w1)
        # 1
        # 2
        # 3

                
    def test(self, X):
        
        test_freq = self.create_table(X, self._dict)
        
        correct = 0
        p_class_data = np.zeros(len(X.target_names))
        
        
        for target, i in zip(X.target[:3], range(len(X.data[:3]))):
            if(i % 1000 == 0):
                print('Test doc [%d] out of [%d]' % (i, 3))
                

            for cls in range(len(X.target_names)):
                likelihood = 1
                for key, vec in self.freq.items():
                    test_vec = test_freq[key]                 
                    group = vec[np.where(self.freq['class'] == cls)]
                    
                    mean_y = group.mean()
                    var_y = group.var()
                    if test_vec[i] != 0 and mean_y !=0 and var_y != 0:
                            
                        p_l = self.likelihood_prob(x=test_vec[i], mean_y=mean_y, var_y=var_y)
                        
                        likelihood = likelihood*p_l
                        
                p_class_data[cls] = self.p_class[cls]*likelihood
        
            
            
            if(np.argmax(p_class_data) == target):
                correct += 1
                
            print(np.argmax(p_class_data), target)
            
            print('Accuracy %f' % (correct / (i+1)))
            
            
            
            

In [4]:
classifier = GNB()


classifier.train(train_data)

classifier.test(test_data)


Dictionary length 85113

Doc [0] out of [11314]
Doc [1000] out of [11314]
Doc [2000] out of [11314]
Doc [3000] out of [11314]
Doc [4000] out of [11314]
Doc [5000] out of [11314]
Doc [6000] out of [11314]
Doc [7000] out of [11314]
Doc [8000] out of [11314]
Doc [9000] out of [11314]
Doc [10000] out of [11314]
Doc [11000] out of [11314]
Doc [0] out of [7532]
Doc [1000] out of [7532]
Doc [2000] out of [7532]
Doc [3000] out of [7532]
Doc [4000] out of [7532]
Doc [5000] out of [7532]
Doc [6000] out of [7532]
Doc [7000] out of [7532]
Test doc [0] out of [3]
0 7
Accuracy 0.000000
0 5
Accuracy 0.000000
0 0
Accuracy 0.333333
